###  Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [4]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [5]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [6]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

In [7]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [8]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [9]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [9]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [10]:
mlp.score(X, Y)

0.68022899006559012

In [11]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.521662
Photography                   0.229354
Architecture & Design         0.111225
Drawings                      0.103381
Painting & Sculpture          0.034377
Name: Department, dtype: float64

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([ 0.57519536,  0.52577072,  0.36922856,  0.48580744,  0.54039799])

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [14]:
# Import the model.
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Establish and fit the model, with 2 layers and 10 perceptron each layer.
mlp = MLPClassifier(hidden_layer_sizes=(10,10))
mlp.fit(X, Y)

mlp.score(X, Y)

0.7452121234474556

In [15]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622548
Photography              0.226397
Architecture & Design    0.113000
Painting & Sculpture     0.033683
Media and Performance    0.004371
Name: Department, dtype: float64

In [16]:
cross_val_score(mlp, X, Y, cv=5)

array([0.64327863, 0.71748652, 0.63602416, 0.63285536, 0.59595168])

**Comments:** With 2 layers with 10 and 10 perceptrons, we get better results than the first option with 1 layer with 1000 perceptrons 

In [17]:
#Let's try with 2 layer withmore neurons

mlp = MLPClassifier(hidden_layer_sizes=(500,250))
mlp.fit(X, Y)

mlp.score(X, Y)

0.8056265041355082

In [18]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622548
Photography              0.226397
Architecture & Design    0.113000
Painting & Sculpture     0.033683
Media and Performance    0.004371
Name: Department, dtype: float64

**Comments:** So far, it is the best result, 2 layers with more perceptrons than the previous one.

In [20]:
#Let's try with 4 layers with 10 neurons

mlp = MLPClassifier(hidden_layer_sizes=(10,10,10,10))
mlp.fit(X, Y)

mlp.score(X, Y)

0.7461618611170021

In [21]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622548
Photography              0.226397
Architecture & Design    0.113000
Painting & Sculpture     0.033683
Media and Performance    0.004371
Name: Department, dtype: float64

In [22]:
cross_val_score(mlp, X, Y, cv=5)

array([0.67176839, 0.6904246 , 0.62892444, 0.61450505, 0.61439506])

**Comments:** Based on the models above, I can say that the number of perceptrons it's as important as the number of layers, depending on the specific case.


_________________

By: Wendy Navarrete

September, 2019